## 데이터 로드

In [1]:
import os
import glob
import random
import pandas as pd
import tiktoken

# 사용자 데이터프레임 로드 (예시 파일 경로 사용)
# tsv 파일을 DataFrame으로 읽기
user = pd.read_csv('data/user.tsv', sep='\t', names=['User', 'History', 'Train', 'Test'])

history_news = pd.read_csv('data/history/news.tsv', sep='\t',  names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'])
train_news = pd.read_csv('data/train/news.tsv', sep='\t', names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'])

# publish 순서에 맞게 오름차순으로 정렬
history_news_sorted = history_news.sort_values(by='Publish', ascending=True).reset_index(drop=True)
train_news_sorted = train_news.sort_values(by='Publish', ascending=True).reset_index(drop=True)

## 프롬프트 생성 함수

In [2]:
def find_similar_news(news_sorted, ids, used_ids):
    """
    Negative 샘플로 사용할 뉴스 기사 찾는 함수 (비슷한 발행일 기준)
    """
    negative_train_ids = []
    negative_train_titles = []

    for id in ids:
        # 주어진 News ID에 해당하는 인덱스 찾기
        idx = news_sorted[news_sorted['News ID'] == id].index[0]
        above_idx, below_idx = idx - 1, idx + 1
        similar_ids = []
        similar_titles = []

        # 이미 존재하는 News ID를 제외하고 위에서 가장 가까운 두 개의 News를 찾기 (이전에 나온 뉴스)
        above_count = 0
        while above_idx >= 0 and above_count < 2:
            if news_sorted.loc[above_idx, 'News ID'] not in used_ids:
                similar_ids.append(news_sorted.loc[above_idx, 'News ID'])
                similar_titles.append(news_sorted.loc[above_idx, 'Title'])
                used_ids.add(news_sorted.loc[above_idx, 'News ID'])
                above_count += 1
            above_idx -= 1

        # 이미 존재하는 News ID를 제외하고 아래에서 가장 가까운 두 개의 News를 찾기 (이후에 나온 뉴스)
        below_count = 0
        while below_idx < len(news_sorted) and below_count < 2:
            if news_sorted.loc[below_idx, 'News ID'] not in used_ids:
                similar_ids.append(news_sorted.loc[below_idx, 'News ID'])
                similar_titles.append(news_sorted.loc[below_idx, 'Title'])
                used_ids.add(news_sorted.loc[below_idx, 'News ID'])
                below_count += 1
            below_idx += 1

        # 필요한 뉴스 수가 4개보다 적을 경우 추가로 위와 아래에서 가져오기
        if len(similar_ids) < 4:
            remaining_needed = 4 - len(similar_ids)
            # 위에서 추가로 가져오기
            while above_idx >= 0 and remaining_needed > 0:
                if news_sorted.loc[above_idx, 'News ID'] not in used_ids:
                    similar_ids.append(news_sorted.loc[above_idx, 'News ID'])
                    similar_titles.append(news_sorted.loc[above_idx, 'Title'])
                    used_ids.add(news_sorted.loc[above_idx, 'News ID'])
                    remaining_needed -= 1
                above_idx -= 1

            # 아직 부족하면 아래에서 추가로 가져오기
            while below_idx < len(news_sorted) and remaining_needed > 0:
                if news_sorted.loc[below_idx, 'News ID'] not in used_ids:
                    similar_ids.append(news_sorted.loc[below_idx, 'News ID'])
                    similar_titles.append(news_sorted.loc[below_idx, 'Title'])
                    used_ids.add(news_sorted.loc[below_idx, 'News ID'])
                    remaining_needed -= 1
                below_idx += 1

        # 최종적으로 negative 샘플 리스트에 추가
        negative_train_ids.append(similar_ids)
        negative_train_titles.append(similar_titles)

    return negative_train_ids, negative_train_titles


def save_user_file(user_data, output_folder_path, purpose):
    """
    Prompt 저장 함수

    """
    
    # 기존 파일들 삭제
    if os.path.exists(output_folder_path):
        for file_path in glob.glob(os.path.join(output_folder_path, "*.txt")):
            os.remove(file_path)
        print(f'[{purpose}] 기존 User Prompts 삭제')    
        
    os.makedirs(output_folder_path, exist_ok=True)

    # user prompt를 각 user별 txt파일로 저장
    for user_id, content in user_data.items():
        file_path = os.path.join(output_folder_path, f"{user_id}.txt")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(content)
            

def save_metadata_file(meta_folder_path, token_counts_and_outputs, user_metadata, purpose):
    """
    metadata 저장 함수

    """

    # 기존 파일들 삭제
    if os.path.exists(meta_folder_path):
        for file_path in glob.glob(os.path.join(meta_folder_path, "*.txt")):
            os.remove(file_path)    
            
    os.makedirs(meta_folder_path, exist_ok=True)

    # metadata 저장 위치
    meta_file_path = os.path.join(meta_folder_path, "output_metadata.txt")

    total_input_tokens = 0
    total_output_count = 0

    # metadata 파일에 user별 token 수, history 수, question 수 등 기록
    with open(meta_file_path, "w", encoding="utf-8") as meta_file:
        for user_id, token_count, output_count, num_news_ids, num_questions in sorted(token_counts_and_outputs, key=lambda x: int(x[0][1:])):
            output_line = (f"User ID: {user_id:<5} Input Tokens: {token_count:<6} Output Tokens: {output_count:<4}  "
                           f"History 수: {num_news_ids:<3}  Question 수: {num_questions}")
            total_input_tokens += token_count
            total_output_count += output_count

            meta_file.write(output_line + "\n")

        # 결과 저장
        total_line = f"\nTotal Input Tokens: {total_input_tokens}\nTotal Output Tokens: {total_output_count}"
        print(f"[{purpose}] {total_line}\n")
        meta_file.write(total_line + "\n")
        

def save_hidden_positions(meta_folder_path, hidden_positions_data):
    """
    question 정답 저장 함수

    """
    hidden_positions_file_path = os.path.join(meta_folder_path, "hidden_positions.txt")
    with open(hidden_positions_file_path, "w", encoding="utf-8") as hidden_file:
        for user_id, positions in hidden_positions_data:
            hidden_file.write(f"{user_id:<5}: {positions}\n")
            

def create(purpose, model, user_count, max_question, save_forder="user_prompts"):
    """
    prompt 생성 main 함수
    
    """
    
    # user자별 input과 output 데이터를 저장할 딕셔너리
    user_data = {}
    user_metadata = []
    hidden_positions_data = []

    # user ID 목록 생성
    user_ids = [f'U{i}' for i in range(1, user_count + 1)]

    for ID in user_ids:

        # user의 history news title 추출
        history = user[user['User'] == ID]['History'].iloc[0]
        news_ids = []
        [news_ids.append(entry.split(',')[0]) for entry in history.split(';') if entry and entry.split(',')[0] not in news_ids]
        titles = []
        for news_id in news_ids:
            matching_rows = history_news[history_news['News ID'] == news_id]
            if not matching_rows.empty:
                titles.append(matching_rows.iloc[0]['Title'])

        
        # user의 train news title 추출
        train = user[user['User'] == ID]['Train'].iloc[0]
        train_ids = []
        [train_ids.append(entry.split(',')[0]) for entry in train.split(';') if entry and entry.split(',')[0] not in train_ids]
        train_titles = []
        for train_id in train_ids:
            matching_rows = train_news[train_news['News ID'] == train_id]
            if not matching_rows.empty:
                train_titles.append(matching_rows.iloc[0]['Title'])

        # History news에 대한 negative news 기사 찾기 (negative용 prompt 생성 시)
        if purpose == "with_negative":
            used_ids = set(news_ids)
            negative_ids, negative_titles = find_similar_news(history_news_sorted, news_ids, used_ids)

        # Train news에 대한 newgative 기사 찾기
        used_ids = set(train_ids)  # 중복을 방지하기 위해 used_ids에 train_id를 추가
        negative_train_ids, negative_train_titles = find_similar_news(train_news_sorted, train_ids, used_ids)
        
        # 질문 생성용 데이터 설정 (Train news와 negative news 결합)
        question_ids = [negative_list + [title] for negative_list, title in zip(negative_train_ids, train_ids)]
        question_titles = [negative_list + [title] for negative_list, title in zip(negative_train_titles, train_titles)]


        # 질문 정답 값의 위치 저장을 위한 리스트 초기화
        hidden_positions = []

        # 각 행을 섞고 정답 값의 위치를 저장
        for row in question_titles:
            hidden_value = row[-1]  # 히든 값 (마지막 요소)
            random.shuffle(row)     # 전체 행을 섞기
            hidden_index = row.index(hidden_value)     # 히든 값의 새로운 위치 찾기
            hidden_positions.append(hidden_index + 1)  # 히든 값의 위치 저장

        # 정답 위치 데이터 추가
        hidden_positions_data.append((ID, hidden_positions))
        number = len(titles)

        # prompt text 생성
        user_content = ""
        if purpose == "only_positive":
            user_content += "[Click History]\n"
            for i in range(number):
                user_content += f"{i+1}) click : {titles[i]}\n"

            user_content += "\n[Questions]\nRank the five candidate news for each question based on the user's news interests.\n"
            for i, titles in enumerate(question_titles[:max_question]):
                user_content += f"Question {i + 1}) "
                user_content += " / ".join([f"{j + 1}: {title}" for j, title in enumerate(titles)]) + "\n"
        elif purpose == "with_negative":
            user_content += "[News of interest to the user]\n"
            for i in range(number):
                combined_titles = [titles[i]] + negative_titles[i]
                random.shuffle(combined_titles)
                user_content += f"{i + 1}) " + " / ".join(combined_titles) + "\n"
                user_content += f"Of the five news above, the news that the user is most interested in : {titles[i]}\n\n"

            user_content += "[Questions]\nRank the five candidate news for each question based on the user's news interests.\n"
            for i, titles in enumerate(question_titles[:max_question]):
                user_content += f"Question {i + 1}) "
                user_content += " / ".join([f"{j + 1}: {title}" for j, title in enumerate(titles)]) + "\n"

        user_content += "\nDon't explain why in your answer, just list news articles ranked for each question.\n"
        
        # user prompt를 딕셔너리에 저장
        user_data[ID] = user_content
        user_metadata.append((ID, number, len(question_titles)))

    # user prompt 파일 저장
    output_folder_path = f"{save_forder}/{purpose}"
    save_user_file(user_data, output_folder_path, purpose)

    # 각 user별 token 수와 output 값을 저장할 리스트
    token_counts_and_outputs = []

    # tiktoken을 사용하여 토큰 수 계산
    encoding = tiktoken.encoding_for_model(model)

    # 모든 텍스트 파일에 대해 반복 수행
    for file_path in sorted(glob.glob(os.path.join(f"{save_forder}/{purpose}", "*.txt"))):
        with open(file_path, "r", encoding="utf-8") as file:
            # 파일에서 input 텍스트 추출
            input_text = file.read()

            # 토큰 수 계산
            token_count = len(encoding.encode(input_text))

            # 사용자 ID 추출
            user_id = os.path.basename(file_path).split(".")[0]

            # Question 수에 따른 output count 계산
            num_questions = input_text.count("Question ")
            output_count = 18 + (num_questions - 1) * 19 if num_questions > 0 else 0

            # 사용자 메타데이터에서 number와 num_questions 가져오기
            # number = next((item[1] for item in user_metadata if item[0] == user_id),0)
            number = next(item[1] for item in user_metadata if item[0] == user_id)

            # 사용자 ID, 토큰 수, output count, news_ids 수, question 수를 리스트에 추가
            token_counts_and_outputs.append((user_id, token_count, output_count, number, num_questions))

    
    # metadata 파일 저장
    meta_folder_path = f"{save_forder}/{purpose}/metadata"
    save_metadata_file(meta_folder_path, token_counts_and_outputs, user_metadata, purpose)

    # question 정답 위치 저장
    save_hidden_positions(meta_folder_path, hidden_positions_data)


## 실행
[create 함수]
- purpose = 어떤 목적으로 prompt를 생성할 것인지  [only_positive / with_negative]   
- model = 사용할 gpt (token 계산 용도)  [gpt-40-mini / gpt-3.5-turbo]
- user_count = 몇 명의 user prompt를 생성할 것인지
- max_question = 최대 질문 수
- save_forder = 결과를 저장할 폴더 이름

In [3]:
create(purpose='only_positive', model="gpt-4o-mini", user_count=1500, max_question=30, save_forder="user_prompts")
create(purpose='with_negative', model="gpt-4o-mini", user_count=1500, max_question=30, save_forder="user_prompts")

[only_positive] 기존 User Prompts 삭제
[only_positive] 
Total Input Tokens: 2233365
Total Output Tokens: 364003

[with_negative] 기존 User Prompts 삭제
[with_negative] 
Total Input Tokens: 4263281
Total Output Tokens: 364003

